# Connect to Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


##move to dataset directory

In [ ]:
%cd '/content/gdrive/MyDrive/Semester 5/Capstone Project/Dataset/NLP Dataset'

/content/gdrive/MyDrive/Semester 5/Capstone Project/Dataset/NLP Dataset


# Import Library NLP and CNN

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
nltk.download("punkt")
import json
import pickle


import numpy as np
import random
from keras.models import Sequential
from keras.layers import Activation,Dropout,Dense
from keras.optimizers import SGD
from keras.models import load_model

#chatbot gui
from termcolor import colored

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


##check a directory

In [ ]:
ls

 1.ipynb   chatbot_model.h5  'dataset 2'/   intents.json  'new dataset'/             words.pkl
 4.ipynb   classes.pkl        intens.json   Intents.json  'Untitled document.gdoc'


#Inisialisasi variabel

In [ ]:
words=[]
documents=[]
classes=[]
ignore_words=['?','!','.','“', '”','(', ')', ',','’']

# baca file json
data_file = open('intents.json',)
intents=json.load(data_file)

##Ekstrak dataset

In [ ]:
nltk.download('wordnet')

# Loop untuk mengekstrak kata, dokumen, dan kelas dari file intents JSON
for intent in intents['intents']:
  for pattern in intent['patterns']:
    w=nltk.word_tokenize(pattern)
    words.extend(w)

    documents.append((w,intent['tag']))
     # Tambahkan ke daftar kelas jika belum ada
    if intent['tag'] not in classes:
      classes.append(intent['tag'])

#Lematisasi dan ubah huruf kecil, serta hapus duplikat kataes
words=[lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words=sorted(list(set(words)))

# Urutkan kelas
classes = sorted(classes, key=str)

#decuments= pattern , intents
print(len(documents),"documents")
#classes= intents
print(len(classes),"classes",classes)
#words = each words , vocabulary
print(len(words),"unique lemmatized words",words)

# Simpan kata dan kelas dalam file pickle
pickle.dump(words,open('words2.pkl','wb'))
pickle.dump(classes,open('classes2.pkl','wb'))


975 documents
400 classes ['EyeU', ['EyeU'], ['EyeU'], ['EyeU'], ['EyeU'], ['EyeU'], ['alergi mata'], ['alergi mata'], ['alergi mata'], ['alergi mata'], ['alkohol'], ['alkohol'], ['alkohol'], ['anak-anak katarak'], ['anak-anak katarak'], ['anak-anak katarak'], ['anak-anak katarak'], ['aplikasi diagnosis'], ['aplikasi diagnosis'], ['aplikasi diagnosis'], ['aplikasi'], ['aplikasi'], ['aplikasi'], ['aplikasi'], ['cahaya'], ['cahaya'], ['cahaya'], ['cegah alergi'], ['cegah alergi'], ['cegah alergi'], ['cegah glaukoma'], ['cegah glaukoma'], ['cegah glaukoma'], ['diabetes'], ['diabetes'], ['diabetes'], ['diagnosis glaukoma'], ['diagnosis glaukoma'], ['diagnosis glaukoma'], ['diagnosis glaukoma'], ['diagnosis katarak'], ['diagnosis katarak'], ['diagnosis katarak'], ['diagnosis katarak'], ['edukasi'], ['edukasi'], ['edukasi'], ['faktor resiko'], ['faktor resiko'], ['faktor resiko'], ['gejala awal katarak'], ['gejala awal katarak'], ['gejala awal katarak'], ['gejala awal katarak'], ['gejala awa

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Split Dataset

In [ ]:
# Buat data latih menggunakan metode bag of words
training=[]
output_=[0]*len(classes)


#bag of words algorithm
for doc in documents:
    #intialize bag for each word
    bag=[]

    pattern_word=doc[0]

    # Lematisasi dan ubah huruf kecil pada setiap kata dalam pola
    pattern_word=[lemmatizer.lemmatize(word.lower()) for word in pattern_word]
    # Metode bag of words
    for w in words:
      bag.append(1) if w in pattern_word else bag.append(0)

    #output
    output_row=list(output_)
    output_row[classes.index(doc[1])]=1

    training.append([bag,output_row])

# Acak dan ubah menjadi array
random.shuffle(training)
training=np.array(training)

#train_x is input and train_y is output
train_x=list(training[:,0])
train_y=list(training[:,1])

print('training data created')


training data created


<ipython-input-10-017c9274813f>:29: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training=np.array(training)


# Build a Model

In [ ]:
# Buat model sequential dengan 3 layer

model=Sequential()
#first layer with 128 neurons
model.add(Dense(128,input_shape=(len(train_x[0]),),activation='relu'))
model.add(Dropout(0.5))
#second layer with 64 neurons
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
#output layer
model.add(Dense(len(train_y[0]),activation='softmax'))

# Metode stochastic gradient descent
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
#compiling model
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])

#fitting model
hist=model.fit(np.array(train_x),np.array(train_y),epochs=200,batch_size=5,verbose=1,validation_split=0.2)
#saving model
model.save('chatbot_model2.h5',hist)

print('model created')

Epoch 1/200
156/156 [==============================] - 6s 5ms/step - loss: 5.7253 - accuracy: 0.0731 - val_loss: 5.2159 - val_accuracy: 0.0615
Epoch 2/200
156/156 [==============================] - 1s 4ms/step - loss: 4.6064 - accuracy: 0.0923 - val_loss: 4.2518 - val_accuracy: 0.0667
Epoch 3/200
156/156 [==============================] - 1s 4ms/step - loss: 3.8713 - accuracy: 0.1333 - val_loss: 3.4691 - val_accuracy: 0.1436
Epoch 4/200
156/156 [==============================] - 1s 4ms/step - loss: 3.2353 - accuracy: 0.2077 - val_loss: 2.8664 - val_accuracy: 0.2462
Epoch 5/200
156/156 [==============================] - 1s 4ms/step - loss: 2.8232 - accuracy: 0.2538 - val_loss: 2.4249 - val_accuracy: 0.3846
Epoch 6/200
156/156 [==============================] - 1s 4ms/step - loss: 2.3994 - accuracy: 0.3244 - val_loss: 1.9813 - val_accuracy: 0.4872
Epoch 7/200
156/156 [==============================] - 1s 4ms/step - loss: 2.1341 - accuracy: 0.4000 - val_loss: 1.5897 - val_accuracy: 0.6359

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
#loading words and classes files

words=pickle.load(open("words2.pkl","rb"))
classes=pickle.load(open("classes2.pkl","rb"))

#Run a Chatbot

In [ ]:
# Fungsi membersihkan kalimat
def clean_up_sentence(sentence):
  #toknize sentence
  sentence_words=nltk.word_tokenize(sentence)
  #lemmtaized and lower each word
  sentence_words=[lemmatizer.lemmatize(word.lower()) for word in sentence_words]

  return sentence_words

def bow(sentence,words,show_details=False):
  #cleaning up sentence
  sentence_words=clean_up_sentence(sentence)

  bag=[0]*len(words)

  for s in sentence_words:
    for i,w in enumerate(words):
      if w==s:
        # bag equal one if current word is in the vocabulary position
        bag[i]=1

        if show_details==True:
          print('Found in %s',w)

  return np.array(bag)

In [ ]:
# Fungsi untuk memprediksi kelas
def predict_class(sentence,model):
  #filter prediction that below threshold
  p=bow(sentence,words,show_details=False)
  res=model.predict(np.array([p]))[0]

  ERROR_THRESHOLD=0.25

  results=[[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]

  #sort by strong probability
  results.sort(key=lambda x:x[1],reverse=True)

  return_list=[]
  for r in results:
      return_list.append({"intent":classes[r[0]],"probability":str(r[1])})


  return return_list


In [ ]:
# Fungsi untuk mendapatkan respons berdasarkan intent
def getResponse(ints,intents_json):
  tag=ints[0]['intent']
  list_intents=intents_json['intents']

  for i in list_intents:

    if (tag==i['tag']):
      result=random.choice(i['responses'])
      break

  return result


def chatbot_response(msg):
  ints=predict_class(msg,model)
  res=getResponse(ints,intents)

  return res

In [ ]:
# Fungsi untuk menjalankan
def run():
  print(colored("EyeU CHATBOT", 'red'))
  print("")
  while(1):
     request=input("")
     response=chatbot_response(request)
     print(colored(response, 'cyan'))
     if request=="tidak"or request== "tidak ada" or request=="See you later" or request== "Goodbye" or request=="Nice chatting to you, bye" or request=="Till next time":
           break

In [ ]:
run()

EyeU CHATBOT

gejala awal katarak
1/1 [==============================] - 0s 55ms/step
Umumnya, gejala awal katarak termasuk penglihatan kabur, kesulitan melihat detail, dan perlu penerangan lebih saat membaca.


KeyboardInterrupt: ignored